In [1]:
import json
import pandas as pd
import re

from pathlib import Path

In [2]:
df = pd.read_json("./results/multinli_transforms_bertscore_dev_matched.jsonl", lines=True)
df.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,style,sentence1_transform1,sentence1_transform2,sentence2_transform1,sentence2_transform2,sentence1_transform1_bertscore,sentence1_transform2_bertscore,sentence2_transform1_bertscore,sentence2_transform2_bertscore
0,"[neutral, entailment, neutral, neutral, neutral]",slate,neutral,63735n,63735,The new rights are nice enough,( ( The ( new rights ) ) ( are ( nice enough )...,(ROOT (S (NP (DT The) (JJ new) (NNS rights)) (...,Everyone really likes the newest benefits,( Everyone ( really ( likes ( the ( newest ben...,(ROOT (S (NP (NN Everyone)) (VP (ADVP (RB real...,pir,"Arrr, thar be some fine new rights fer us!","'Tis a pleasant sight, these new rights, me he...","Arrr, me hearties be lovin' the latest booty.",All hands on deck be happy with the brand new ...,0.835507,0.846157,0.847767,0.878438
1,"[contradiction, contradiction, contradiction, ...",government,contradiction,91383c,91383,This site includes a list of all award winners...,( ( This site ) ( ( includes ( ( ( ( a list ) ...,(ROOT (S (NP (DT This) (NN site)) (VP (VBZ inc...,The Government Executive articles housed on th...,( ( ( The ( Government ( Executive articles ) ...,(ROOT (S (NP (NP (DT The) (NNP Government) (NN...,pir,This site be includin' a list o' all award win...,"Arrr, this here site be havin' a list o' all t...",Ye Government Executive articles housed on tha...,Thar Government Executive pieces housed on th'...,0.938552,0.870573,0.949286,0.898377
2,"[entailment, entailment, entailment, entailmen...",telephone,entailment,755e,755,uh i don't know i i have mixed emotions about ...,( ( ( ( uh ( i ( ( do n't ) ( know ( ( i i ) (...,(ROOT (SINV (S (S (INTJ (UH uh)) (NP (FW i)) (...,"I like him for the most part, but would still ...",( I ( ( ( ( ( ( like him ) ( for ( the ( most ...,(ROOT (S (NP (PRP I)) (VP (VP (VBP like) (NP (...,pir,"Arrr, I be not sure me hearties, sometimes I f...","Aye, me hearty, me feelings be mixed about him...","I be likin' him for the most part, but I'd sti...","I be fancyin' him for the most part, but I'd s...",0.873443,0.876103,0.920837,0.929856
3,"[contradiction, contradiction, contradiction, ...",telephone,contradiction,78013c,78013,yeah i i think my favorite restaurant is alway...,( yeah ( ( i i ) ( think ( ( my ( favorite res...,(ROOT (S (VP (VB yeah) (NP (NP (FW i) (FW i)) ...,My favorite restaurants are always at least a ...,( ( My ( favorite restaurants ) ) ( ( ( ( are ...,(ROOT (S (NP (PRP$ My) (JJ favorite) (NNS rest...,pir,"Aye, me favorite galley be always the one near...","Arr, methinks me top tavern's always been the ...",Me favorite taverns be always at least a hundr...,My preferred grub spots be a hundred leagues o...,0.869231,0.845367,0.933945,0.901702
4,"[contradiction, contradiction, contradiction, ...",telephone,contradiction,96377c,96377,i don't know um do you do a lot of camping,( i ( ( do n't ) ( know ( um ( do ( you ( do (...,(ROOT (S (NP (FW i)) (VP (VBP do) (RB n't) (VP...,I know exactly.,( I ( ( know exactly ) . ) ),(ROOT (S (NP (PRP I)) (VP (VBP know) (ADVP (RB...,pir,"I don't be knowin' matey, do ye partake in a l...","Arrr, I be clueless, but do yer fancy a lot o'...",I savvy precisely.,I be knowin' precisely.,0.874817,0.852997,0.909285,0.882909


In [17]:
df_filter_1 = df[abs(df['sentence1_transform1_bertscore'] - df['sentence1_transform2_bertscore']) > 0.1]
df_filter_2 = df[abs(df['sentence2_transform1_bertscore'] - df['sentence2_transform2_bertscore']) > 0.1]
print(max(abs(row['sentence1_transform1_bertscore'] - row['sentence1_transform2_bertscore']) for index, row in df.iterrows()))
print(max(abs(row['sentence2_transform1_bertscore'] - row['sentence2_transform2_bertscore']) for index, row in df.iterrows()))

0.12326139210000009
0.11731135849999996


In [22]:
for index, row in df_filter_1.iterrows():
    print(row['sentence1'])
    print(f"{row['sentence1_transform1']}, ({row['sentence1_transform1_bertscore']})")
    print(f"{row['sentence1_transform2']}, ({row['sentence1_transform2_bertscore']})")
    print()

Savonarola burned in Florence
Savonarola be burned in Florence., (0.9661374688000001)
Florence be witnessin' the burnin' o' Savonarola., (0.8428760767)

 9th circa b.c.First signs of pre-Roman Etruscans
9th circa b.c. First signs o' pre-Roman Etruscans, (0.9821109772000001)
9th year in th' cirrrrrrrrca. Signs o' pre-Roman Etruscans be showin' up., (0.8649559617)

you sound like this girl that i talked to about books and we got into movies one night
Ye sound like this lass that I talked to about books an' we got into movies one night., (0.9636355042)
Yer voice be remindin' me of a fine lass I once talked to 'bout books, but soon we be chattin' 'bout the flicks., (0.8606296182000001)

Annette told me how you'd escaped.
Annette told me how ye'd escaped, matey., (0.9541797638)
Ahoy, me heartie! Annette be telling me how ye be escapin'., (0.8418803811000001)

aCondition Assessment Survey (CAS).
a Condition Assessment Survey (CAS) be what we be needin'!, (0.9070871472)
Arrr! A CAS be require

In [21]:
# df_filter_2[['sentence2', 'sentence2_transform1', 'sentence2_transform2', 'sentence2_transform1_bertscore', 'sentence2_transform2_bertscore']].head(n=10)
for index, row in df_filter_2.iterrows():
    print(row['sentence2'])
    print(f"{row['sentence2_transform1']}, ({row['sentence2_transform1_bertscore']})")
    print(f"{row['sentence2_transform2']}, ({row['sentence2_transform2_bertscore']})")
    print()

Telecommuters buy generic office space.
Telecommuters be buyin' plain office quarters., (0.9352453947)
Arrr, them who work from afar be purchasin' nothin' but ordinary workspace., (0.8188115954)

You were thought to be dead!
Ye were thought to be dead!, (0.9979718328)
Yer supposed to be shark bait!, (0.8806604743)

At a later date the company informed us they canceled the program.
At a later date the company told us they be cancelin' the program., (0.9672935009)
Aft a wee bit o' time, the company informed us they be swashbucklin' th' program., (0.8613722920000001)

You hate music, don't you? 
Ye hate music, don't ye?, (0.9778265357)
Ahoy, ye despise music, aye?, (0.8646695614000001)

Writing that alternates directions on each line is called boustrophedon.
Writin' that alternates directions on each line be called boustrophedon, me hearties., (0.9377823472000001)
Boustrophedon be the fancy name for writin' that be goin' back and forth on every line, me mateys., (0.8240621090000001)

The 